In [ ]:
from pyaxis import pyaxis

def px_reder(path_raw):
    files = os.listdir(path_raw)
    px_files_name = [ f.replace('.px','') for f in files if f.endswith('.px')]
    px_end ='.px'
    df_dict = {}
    for name in px_files_name:
        px = px = pyaxis.parse(path_raw+name+px_end, encoding='ISO-8859-15')
        df_dict[name] = (px['DATA'],px['METADATA'])
    return df_dict

def get_column_names_except(df, excluded_columns):  
    all_columns = df.columns.tolist()
    included_columns = [col for col in all_columns if col not in excluded_columns]
    return included_columns

def remove_rows_by_values(df, column, values):
    return df[~df[column].isin(values)]

def split_col_px(df_dict):
    dic = {}
    for key, (df, meta) in df_dict.items():
        if 'Jahr' in df.columns:
            col_mask = df.columns.str.contains('Gemeinde (......)', case=False, regex=False)

            if not (any(col_mask) or ('Gemeinde' in df.columns)):
                print(key, 'to break')
                break
                
            if any(col_mask):                
                # Auswahl der Spalten basierend auf der Liste der Booleschen Werte
                selected_cols = df.iloc[:, col_mask]
                # Anwenden der `startswith()`-Methode auf jede Spalte
                mapping = selected_cols.apply(lambda x: x.str.startswith('.....'))
                # Entfernen von unerwünschten Zeichen aus der ausgewählten Spalte
                gemeinde_col = selected_cols.iloc[:, 0].str.replace('\.+', '', regex=True)
                gemeinde_col= gemeinde_col.str.split(' ', n=1, expand=True)
                df[['Gemeindecode', 'Gemeindename']] = gemeinde_col
                df = df.drop(selected_cols.columns.to_list(),axis=1)
                
              
    
            if 'Gemeinde' in df.columns:               
                df[['Gemeindecode', 'Gemeindename']] = df['Gemeinde'].str.split(' ', n=1, expand=True)                
                df.drop(['Gemeinde'],inplace=True,axis=1)     
            print (key)
            
            df.dropna(inplace=True)
            df = remove_rows_by_values(df,'Gemeindecode',['<<','-'])
            df['Gemeindecode'] = df['Gemeindecode'].astype(int)
            df['Jahr'] = df['Jahr'].astype(int)
            flip_col = get_column_names_except(df, ['Gemeindecode', 'Jahr','Gemeindename','DATA'])          
            df = df.pivot(index=['Gemeindecode','Jahr'],columns=flip_col, values='DATA')
            #df = df.apply(pd.to_numeric, errors='coerce')
            dic[key] = df#, meta
            

    return dic

df_dict = px_reder(path_px)
df_dict = split_col_px(df_dict)


In [ ]:
def check_string_length(str1, str2):
    """
    Überprüft, ob die Längen von zwei Strings eine maximale Abweichung von 3 Buchstaben haben.
    :param str1: Erster String
    :param str2: Zweiter String
    :return: True, wenn die Längen der Strings eine maximale Abweichung von 3 Buchstaben haben, andernfalls False.
    """
    length_diff = abs(len(str1) - len(str2))  # Berechnet den Unterschied in der Länge der Strings
    if length_diff <= 3:  # Überprüft, ob die maximale Abweichung von 3 Buchstaben überschritten wird
        return True
    else:
        return False
def create_partial_duplicates(dict_obj):
    """
    Erstellt Teil-Duplikate von Dictionary-Keys, wenn ein Schlüssel zu 50% übereinstimmt.
    :param dict_obj: Ein Dictionary-Objekt
    :return: Eine Liste von Teil-Duplikaten von Dictionary-Keys
    """
    all_keys = dict_obj.keys()  # Extrahiert alle Schlüssel aus dem Dictionary-Objekt
    partial_duplicates = []
    for i, key1 in enumerate(all_keys):
        for j, key2 in enumerate(all_keys):
            if i != j and key1 in key2 and  check_string_length(key1,key2):
                partial_duplicates.append((key1,key2))  # Fügt Teil-Duplikat hinzu

    return sorted(list(set(partial_duplicates)))


duplicates_names = create_partial_duplicates(dict_merge)